I want to have 

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
# List only files in the current directory
files = [f for f in os.listdir('.') if os.path.isfile(f)]
for f in files:
    print(f)

Bandi.jpg
EUDC2024tab.csv.xlsx
EUDC2024tab.txt
FileRenamer.ipynb
README.md
Tab2Database.ipynb
~$EUDC2024tab.csv.xlsx


# Start: Goal
##### Debate Project
The goal of this project is to use programming to enhance the level of competitive academic debating, mostly by doing advanced data-analysis and further techniques.

##### Tabdata
A huge chapter in this is the creation of a comprehensive debate database, that includes all elements of debating that can be used to measure, evaluate, and increase performance. At this junction of the project the **TabData** project tries to set up the database of debates and some follow-up metrics, such as "elo-score". 

##### Tabdata useage
In ideal case this can be used for predicting winners of debates, predicting and analyzing biases and outliers with motions, judges, speaker-scores of certain judges, hardships with motion-types, correlations between outcomes in debate tournaments, or just providing an overview about the long-term trends within debating by having a comprehensive data through a long cross-section of time. 

# About the data
Our data arrives in a csv. <span style="color:red">**At this test programme, it is manually downloaded**</span>  but for future tournament databases, it will be downloaded by scraping with Selenium.

The data consists the outcomes of debate tournaments. This has 3 main sections. 

#### Team tab ####
The teamtab consists the teams that participated at the tournament, and the ranking between the teams at the end of the tournament's preliminary rounds. As a rule of thumb, tournaments run for 5 rounds, but major tournaments such as European or World Championships have 9 preliminary rounds. The top teams go through to the elimination rounds, to reach, and eventually win the finals. Teams that do not reach the playoffs, are ranked by their respective positions in the tab. 

For example, if there are 40 teams in a tournament, the top 8 will go to the semi-finals, with a preset seeding in British Parliamentary debates where in Semi-Final-1 the 1st, 4th, 5th, and 8th seeded teams debate, and in Semi-Final-2 the 2nd, 3rd, 6th, 7th seeded teams debate. Each semi-final will have 2 teams that advance to the finals. So, despite teams do not carry the resulst to the outrounds, they have an impact on each team's result. Teams 9-40 are ranked based on their positions within the tab. 

#### Speaker tab ####
In the same csv, following the teamtab, comes the speakertab. While the teamtab is ranking teams, the speakertab ranks the individual speakers within a debate tournaments. Because in British Parliamentary debating the speaker score of the individual speakers within a debate is summed, and teams are ranked in the order of the sum of these scores, teamtab and speakertab strongly correlates, but they don't have to be the same. Furthermore, it provides further ways to analyse data for us. Both the teamtab and the spakertab is with their own headers within the data, therefore slicing up the csv to teamtab and speakertab is not particularly complex.

#### Debate results ####
The third, and largest part of the csv consists each team's result within a given debate, and also the role they had (f.e. Opening Government) and the adjucating judges too. Each row consists the data for each individual team's separate round. 

Each round was obtained separately and attached to the csv, so every round has its own headers, that is needed, because the data does not consist the roundnumbers themselves.

The data looks something like this: 
| team              | result | side    | adjudicators  |
| :---------------- | :------: | ----: | --------------|
| Team1        |  1st   | Opening Government | Judge1,Judge2 |  
| Team2           |  1st   | Opening Government | Judge3 |
| Team3    |  2nd   | Closing Opposition |  Judge4,Judge6 |
| Team4 |  4th   | Opening Opposition | Judge3 |

From the data we can see that Team 2 and Team 4 debated in the same debate at the given round, because they had the same adjudicators. (Judge 3). *Team2* finished 2nd, and *Team4* finished 4th. This itself is not new information, because we could retrieve this information from the teamtab. However, the teamtab provides no information on the opponents, and the speaker scores the team have received. 


#### The task #### 
The teamtab, the speakertab, and the debate_results dataframes are connected by 1 common column: the team's names. Each tournament, each team has its unique name. However, it is possible for John Blue to run in the team "Blue Brothers" in 1 tournament, with his sibling Adam Blue, and run with a different teamname at a following tournament, with a different teammate, or the very same. Therefore, the hardest part once the database is created is to clear out the database from people with similar names. Fortunately, most of the times debaters have unique names. As a matter of fact, the teamtab is not even needed for the creation of the "debate-level" dataframe. 

So, for each row of debate results, we are first going to retrieve the speaker scores and the speakers for the team from the speakertab. Then we will group the 4 debating teams together to one row. And finally, we will separate each adjudicator to a different row. This will give us 1 row of data in our phenomenal database.

## Start: Read in data ## 

In [4]:
comp_name = 'EUDC2024'
df = pd.read_csv('EUDC2024tab.txt', delimiter='\t')
print(len(df))
df

2300


,Rk,team,categories,R1,R2,R3,R4,R5,R6,R7,R8,R9,Pts,Spk,1sts,2nds,Irons,3rds
0,1,HUJI HY,NaN,1st,1st,1st,1st,1st,1st,2nd,1st,1st,26,1499,8,1,0.0,0.0
1,2,Manchester A,NaN,3rd,1st,1st,1st,1st,2nd,3rd,1st,2nd,21,1473,5,2,0.0,2.0
2,3,LSE B,ESL,2nd,1st,1st,1st,2nd,4th,1st,2nd,1st,21,1452,5,3,0.0,0.0
3,4,Oxford A,NaN,1st,1st,1st,4th,1st,1st,1st,3rd,3rd,20,1485,6,0,0.0,2.0
4,5,Zagreb A,ESL,2nd,2nd,1st,1st,2nd,2nd,1st,4th,1st,20,1483,4,4,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2295,team,result,side,adjudicators,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2296,Copenhagen A,eliminated,Opening Government,"Ilija IvaniševićⒸ , Alishba Irfan , Andrew Che...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2297,LSE B,eliminated,Closing Government,"Ilija IvaniševićⒸ , Alishba Irfan , Andrew Che...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2298,Tel Aviv A,eliminated,Opening Opposition,"Ilija IvaniševićⒸ , Alishba Irfan , Andrew Che...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
df[1000:2000]

,Rk,team,categories,R1,R2,R3,R4,R5,R6,R7,R8,R9,Pts,Spk,1sts,2nds,Irons,3rds
1000,Bucharest C,4th,Closing Government,View Ballot,"Shireen AliⒸ , Nina McCartney , Tom Kuson , Ch...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1001,Sheffield SJ,3rd,Opening Government,View Ballot,"Shireen AliⒸ , Nina McCartney , Tom Kuson , Ch...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1002,Zurich A,2nd,Closing Opposition,View Ballot,"Shireen AliⒸ , Nina McCartney , Tom Kuson , Ch...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1003,Amsterdam RZ,4th,Closing Government,View Ballot,"Nir KligsbergⒸ , Dominik Bartosz Jasiolek , Ol...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1004,GUU A,3rd,Opening Government,View Ballot,"Nir KligsbergⒸ , Dominik Bartosz Jasiolek , Ol...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,Warwick YZ,2nd,Opening Opposition,View Ballot,Ðorđe “this entire competition could&#x27;ve b...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,UAM A,3rd,Opening Opposition,View Ballot,"Novak CelebicⒸ , Lara Bhakdi , Nikita Tarasov",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997,Munich B,1st,Closing Opposition,View Ballot,"Novak CelebicⒸ , Lara Bhakdi , Nikita Tarasov",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998,Durham B,4th,Closing Government,View Ballot,"Novak CelebicⒸ , Lara Bhakdi , Nikita Tarasov",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## I. Teamtab ##
First, we need to create the teamtab

In [6]:
# Find the index of the row with the known value
stop_index = df[df['Rk'] == 'Rk'].index[0]

In [7]:
stop_index

185

In [8]:
teamtab = df.iloc[:stop_index]
teamtab

,Rk,team,categories,R1,R2,R3,R4,R5,R6,R7,R8,R9,Pts,Spk,1sts,2nds,Irons,3rds
0,1,HUJI HY,NaN,1st,1st,1st,1st,1st,1st,2nd,1st,1st,26,1499,8,1,0.0,0.0
1,2,Manchester A,NaN,3rd,1st,1st,1st,1st,2nd,3rd,1st,2nd,21,1473,5,2,0.0,2.0
2,3,LSE B,ESL,2nd,1st,1st,1st,2nd,4th,1st,2nd,1st,21,1452,5,3,0.0,0.0
3,4,Oxford A,NaN,1st,1st,1st,4th,1st,1st,1st,3rd,3rd,20,1485,6,0,0.0,2.0
4,5,Zagreb A,ESL,2nd,2nd,1st,1st,2nd,2nd,1st,4th,1st,20,1483,4,4,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,181,Haifa B,ESL,4th,3rd,4th,3rd,1st,3rd,4th,3rd,4th,7,1278,1,0,0.0,4.0
181,182,Swing A,NaN,—,—,—,1st,4th,—,—,—,1st,6,450,2,0,0.0,0.0
182,183,Leiden MS,NaN,4th,4th,4th,2nd,4th,3rd,4th,3rd,4th,4,1276,0,1,0.0,2.0
183,184,Liverpool A,NaN,4th,4th,3rd,4th,2nd,4th,4th,4th,3rd,4,1261,0,1,0.0,2.0


In [9]:
#Remove useless data
target_column = 'Spk'
column_index = teamtab.columns.get_loc(target_column)

# Slice the DataFrame to keep columns up to and including the target column
teamtab = teamtab.iloc[:, :column_index + 1]
teamtab


,Rk,team,categories,R1,R2,R3,R4,R5,R6,R7,R8,R9,Pts,Spk
0,1,HUJI HY,NaN,1st,1st,1st,1st,1st,1st,2nd,1st,1st,26,1499
1,2,Manchester A,NaN,3rd,1st,1st,1st,1st,2nd,3rd,1st,2nd,21,1473
2,3,LSE B,ESL,2nd,1st,1st,1st,2nd,4th,1st,2nd,1st,21,1452
3,4,Oxford A,NaN,1st,1st,1st,4th,1st,1st,1st,3rd,3rd,20,1485
4,5,Zagreb A,ESL,2nd,2nd,1st,1st,2nd,2nd,1st,4th,1st,20,1483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,181,Haifa B,ESL,4th,3rd,4th,3rd,1st,3rd,4th,3rd,4th,7,1278
181,182,Swing A,NaN,—,—,—,1st,4th,—,—,—,1st,6,450
182,183,Leiden MS,NaN,4th,4th,4th,2nd,4th,3rd,4th,3rd,4th,4,1276
183,184,Liverpool A,NaN,4th,4th,3rd,4th,2nd,4th,4th,4th,3rd,4,1261


In [10]:
df2 = df.drop(teamtab.index)
df2.reset_index(drop=True, inplace=True)
df2

,Rk,team,categories,R1,R2,R3,R4,R5,R6,R7,R8,R9,Pts,Spk,1sts,2nds,Irons,3rds
0,Rk,name,category,team,R1,R2,R3,R4,R5,R6,R7,R8,R9,Total,Avg,Stdev,NaN,NaN
1,1,Hadar Goldberg,ESL,HUJI HY,83,83,84,83,80,85,85,88,85,756,84.00,2.05,NaN,NaN
2,2,Martha McKinney-Perry,NaN,TCD Hist CM,79,85,83,81,86,80,84,87,80,745,82.78,2.74,NaN,NaN
3,3=,Aidan Woo,NaN,Oxford A,82,83,83,79,83,84,84,84,82,744,82.67,1.49,NaN,NaN
4,3=,Petar Žnidar,ESL,Zagreb A,79,82,80,83,85,81,85,82,87,744,82.67,2.45,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2110,team,result,side,adjudicators,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2111,Copenhagen A,eliminated,Opening Government,"Ilija IvaniševićⒸ , Alishba Irfan , Andrew Che...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2112,LSE B,eliminated,Closing Government,"Ilija IvaniševićⒸ , Alishba Irfan , Andrew Che...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2113,Tel Aviv A,eliminated,Opening Opposition,"Ilija IvaniševićⒸ , Alishba Irfan , Andrew Che...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## II. The speaker tab

In [11]:
stop_index2 = df2[df2['Rk'] == 'team'].index[0]
speakertab = df2.iloc[:stop_index2]

#remove top row & reset index:
speakertab = speakertab.iloc[1:].reset_index(drop=True)
speakertab

,Rk,team,categories,R1,R2,R3,R4,R5,R6,R7,R8,R9,Pts,Spk,1sts,2nds,Irons,3rds
0,1,Hadar Goldberg,ESL,HUJI HY,83,83,84,83,80,85,85,88,85,756,84.00,2.05,NaN,NaN
1,2,Martha McKinney-Perry,NaN,TCD Hist CM,79,85,83,81,86,80,84,87,80,745,82.78,2.74,NaN,NaN
2,3=,Aidan Woo,NaN,Oxford A,82,83,83,79,83,84,84,84,82,744,82.67,1.49,NaN,NaN
3,3=,Petar Žnidar,ESL,Zagreb A,79,82,80,83,85,81,85,82,87,744,82.67,2.45,NaN,NaN
4,5,Yishai Binnes,NaN,HUJI HY,82,83,83,82,78,82,83,87,83,743,82.56,2.17,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,366,Matylda Jellonnek,ESL,Utrecht B,75,77,79,74,74,76,—,—,74,529,75.57,1.76,NaN,NaN
366,367,Alisa Popova,NaN,Copenhagen B,77,70,74,77,—,80,75,74,—,527,75.29,2.91,NaN,NaN
367,368,Speaker 1,NaN,Swing A,—,—,—,76,74,—,—,—,79,229,76.33,2.05,NaN,NaN
368,369,Speaker 2,NaN,Swing A,—,—,—,76,66,—,—,—,79,221,73.67,5.56,NaN,NaN


In [12]:
#Remove useless data
target_column = 'Spk'
column_index = speakertab.columns.get_loc(target_column)

# Slice the DataFrame to keep columns up to and including the target column
speakertab = speakertab.iloc[:, :column_index + 1]
speakertab

,Rk,team,categories,R1,R2,R3,R4,R5,R6,R7,R8,R9,Pts,Spk
0,1,Hadar Goldberg,ESL,HUJI HY,83,83,84,83,80,85,85,88,85,756
1,2,Martha McKinney-Perry,NaN,TCD Hist CM,79,85,83,81,86,80,84,87,80,745
2,3=,Aidan Woo,NaN,Oxford A,82,83,83,79,83,84,84,84,82,744
3,3=,Petar Žnidar,ESL,Zagreb A,79,82,80,83,85,81,85,82,87,744
4,5,Yishai Binnes,NaN,HUJI HY,82,83,83,82,78,82,83,87,83,743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,366,Matylda Jellonnek,ESL,Utrecht B,75,77,79,74,74,76,—,—,74,529
366,367,Alisa Popova,NaN,Copenhagen B,77,70,74,77,—,80,75,74,—,527
367,368,Speaker 1,NaN,Swing A,—,—,—,76,74,—,—,—,79,229
368,369,Speaker 2,NaN,Swing A,—,—,—,76,66,—,—,—,79,221


In [13]:
speakertab = speakertab.rename(columns={'Rk': 'SpeakRank','team':'speaker','R1':'team'})
speakertab

,SpeakRank,speaker,categories,team,R2,R3,R4,R5,R6,R7,R8,R9,Pts,Spk
0,1,Hadar Goldberg,ESL,HUJI HY,83,83,84,83,80,85,85,88,85,756
1,2,Martha McKinney-Perry,NaN,TCD Hist CM,79,85,83,81,86,80,84,87,80,745
2,3=,Aidan Woo,NaN,Oxford A,82,83,83,79,83,84,84,84,82,744
3,3=,Petar Žnidar,ESL,Zagreb A,79,82,80,83,85,81,85,82,87,744
4,5,Yishai Binnes,NaN,HUJI HY,82,83,83,82,78,82,83,87,83,743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,366,Matylda Jellonnek,ESL,Utrecht B,75,77,79,74,74,76,—,—,74,529
366,367,Alisa Popova,NaN,Copenhagen B,77,70,74,77,—,80,75,74,—,527
367,368,Speaker 1,NaN,Swing A,—,—,—,76,74,—,—,—,79,229
368,369,Speaker 2,NaN,Swing A,—,—,—,76,66,—,—,—,79,221


In [14]:
# Get the first 4 and last column names
first_columns = speakertab.columns[:4].tolist()
last_column = speakertab.columns[-1]

# Get the middle columns that need renaming
#These will be the round results
middle_columns = speakertab.columns[4:-1]

# Create new names for the middle columns (e.g., 'NewCol1', 'NewCol2', ...)
new_middle_columns = [f'R{i}' for i in range(1, len(middle_columns) + 1)]

# Combine the first columns, renamed middle columns, and last column
new_columns = first_columns + new_middle_columns + [last_column]

# Apply the new column names to the DataFrame
speakertab.columns = new_columns

speakertab

,SpeakRank,speaker,categories,team,R1,R2,R3,R4,R5,R6,R7,R8,R9,Spk
0,1,Hadar Goldberg,ESL,HUJI HY,83,83,84,83,80,85,85,88,85,756
1,2,Martha McKinney-Perry,NaN,TCD Hist CM,79,85,83,81,86,80,84,87,80,745
2,3=,Aidan Woo,NaN,Oxford A,82,83,83,79,83,84,84,84,82,744
3,3=,Petar Žnidar,ESL,Zagreb A,79,82,80,83,85,81,85,82,87,744
4,5,Yishai Binnes,NaN,HUJI HY,82,83,83,82,78,82,83,87,83,743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,366,Matylda Jellonnek,ESL,Utrecht B,75,77,79,74,74,76,—,—,74,529
366,367,Alisa Popova,NaN,Copenhagen B,77,70,74,77,—,80,75,74,—,527
367,368,Speaker 1,NaN,Swing A,—,—,—,76,74,—,—,—,79,229
368,369,Speaker 2,NaN,Swing A,—,—,—,76,66,—,—,—,79,221


## III. Connecting the speakertab & Debate Results

In [15]:
df3 = df2.drop(speakertab.index)
df3 = df3.iloc[1:].reset_index(drop=True)
df3.reset_index(drop=True, inplace=True)
df3

,Rk,team,categories,R1,R2,R3,R4,R5,R6,R7,R8,R9,Pts,Spk,1sts,2nds,Irons,3rds
0,team,result,side,ballot,adjudicators,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Munich A,1st,Closing Opposition,View Ballot,"Idan BresslerⒸ , Olivia Ortlieb , AnaⓉ",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LSE A,3rd,Opening Government,View Ballot,"Idan BresslerⒸ , Olivia Ortlieb , AnaⓉ",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TCD Phil C,4th,Closing Government,View Ballot,"Idan BresslerⒸ , Olivia Ortlieb , AnaⓉ",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Zagreb A,2nd,Opening Opposition,View Ballot,"Idan BresslerⒸ , Olivia Ortlieb , AnaⓉ",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,team,result,side,adjudicators,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1740,Copenhagen A,eliminated,Opening Government,"Ilija IvaniševićⒸ , Alishba Irfan , Andrew Che...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1741,LSE B,eliminated,Closing Government,"Ilija IvaniševićⒸ , Alishba Irfan , Andrew Che...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1742,Tel Aviv A,eliminated,Opening Opposition,"Ilija IvaniševićⒸ , Alishba Irfan , Andrew Che...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df3.loc[df3['R1'] == 'View Ballot', 'R1'] = df3['R2']
df3 = df3.iloc[:,:4]
df3["Round"] = 0
df3

,Rk,team,categories,R1,Round
0,team,result,side,ballot,0
1,Munich A,1st,Closing Opposition,"Idan BresslerⒸ , Olivia Ortlieb , AnaⓉ",0
2,LSE A,3rd,Opening Government,"Idan BresslerⒸ , Olivia Ortlieb , AnaⓉ",0
3,TCD Phil C,4th,Closing Government,"Idan BresslerⒸ , Olivia Ortlieb , AnaⓉ",0
4,Zagreb A,2nd,Opening Opposition,"Idan BresslerⒸ , Olivia Ortlieb , AnaⓉ",0
...,...,...,...,...,...
1739,team,result,side,adjudicators,0
1740,Copenhagen A,eliminated,Opening Government,"Ilija IvaniševićⒸ , Alishba Irfan , Andrew Che...",0
1741,LSE B,eliminated,Closing Government,"Ilija IvaniševićⒸ , Alishba Irfan , Andrew Che...",0
1742,Tel Aviv A,eliminated,Opening Opposition,"Ilija IvaniševićⒸ , Alishba Irfan , Andrew Che...",0


In [17]:
round_counter = 0
row_numbers = df3.index[df3["Rk"] == "team"].tolist()

#Trick "dumbass": In order for the next for loop to iterate through, I append a 0 to the end of my list. 
#This is needed, so when the loop is at the last round of the we don't end up with referring to a roundnumber, that would be 
#Larger than the last element in the list. 
#What I mean: check a few lines down
row_numbers.append(0)
print(row_numbers)


df4 = df3
for i in df4.index:
    if row_numbers[round_counter] == i:
        print(i)
        print(round_counter)
        
        #Trick "dumbass" --> in the last round if I did not append a 0, row_umbers[round_counter] would be
        #larger than the last element's position in the row_numbers list
        #But things just work perfectly this way, like a shitty spagetthi
        round_counter += 1
    df4.loc[i, "Round"] = round_counter  # Example logic
row_numbers = row_numbers[:-1]
df4

[0, 185, 370, 555, 740, 925, 1110, 1295, 1480, 1665, 1682, 1699, 1708, 1713, 1730, 1739, 0]
0
0
185
1
370
2
555
3
740
4
925
5
1110
6
1295
7
1480
8
1665
9
1682
10
1699
11
1708
12
1713
13
1730
14
1739
15


,Rk,team,categories,R1,Round
0,team,result,side,ballot,1
1,Munich A,1st,Closing Opposition,"Idan BresslerⒸ , Olivia Ortlieb , AnaⓉ",1
2,LSE A,3rd,Opening Government,"Idan BresslerⒸ , Olivia Ortlieb , AnaⓉ",1
3,TCD Phil C,4th,Closing Government,"Idan BresslerⒸ , Olivia Ortlieb , AnaⓉ",1
4,Zagreb A,2nd,Opening Opposition,"Idan BresslerⒸ , Olivia Ortlieb , AnaⓉ",1
...,...,...,...,...,...
1739,team,result,side,adjudicators,16
1740,Copenhagen A,eliminated,Opening Government,"Ilija IvaniševićⒸ , Alishba Irfan , Andrew Che...",16
1741,LSE B,eliminated,Closing Government,"Ilija IvaniševićⒸ , Alishba Irfan , Andrew Che...",16
1742,Tel Aviv A,eliminated,Opening Opposition,"Ilija IvaniševićⒸ , Alishba Irfan , Andrew Che...",16


In [18]:
df4.head(20)

,Rk,team,categories,R1,Round
0,team,result,side,ballot,1
1,Munich A,1st,Closing Opposition,"Idan BresslerⒸ , Olivia Ortlieb , AnaⓉ",1
2,LSE A,3rd,Opening Government,"Idan BresslerⒸ , Olivia Ortlieb , AnaⓉ",1
3,TCD Phil C,4th,Closing Government,"Idan BresslerⒸ , Olivia Ortlieb , AnaⓉ",1
4,Zagreb A,2nd,Opening Opposition,"Idan BresslerⒸ , Olivia Ortlieb , AnaⓉ",1
5,Reichman B,3rd,Opening Government,"Eleanor Douglas 👻Ⓒ , Ahmad Tousif Jami , Azzar...",1
6,UCD L&amp;H A,1st,Opening Opposition,"Eleanor Douglas 👻Ⓒ , Ahmad Tousif Jami , Azzar...",1
7,Porto A,4th,Closing Opposition,"Eleanor Douglas 👻Ⓒ , Ahmad Tousif Jami , Azzar...",1
8,UCD L&amp;H D,2nd,Closing Government,"Eleanor Douglas 👻Ⓒ , Ahmad Tousif Jami , Azzar...",1
9,Cardiff B,4th,Opening Government,"Katie KielyⒸ , Sahil Khandelwal , Lucy HindmarshⓉ",1


In [19]:
df4.tail(20)

,Rk,team,categories,R1,Round
1724,Trivium B,advancing,Opening Government,"Marina KojićⒸ , Jack Palmer , Tom Kuson , Vlad...",14
1725,Tartu A,eliminated,Opening Opposition,"Marina KojićⒸ , Jack Palmer , Tom Kuson , Vlad...",14
1726,Vienna A,eliminated,Closing Government,"Jack Williams 🎮Ⓒ , Brian Kam , Eleanor Douglas...",14
1727,LSE B,advancing,Opening Government,"Jack Williams 🎮Ⓒ , Brian Kam , Eleanor Douglas...",14
1728,Tel Aviv A,advancing,Closing Opposition,"Jack Williams 🎮Ⓒ , Brian Kam , Eleanor Douglas...",14
1729,Haifa A,eliminated,Opening Opposition,"Jack Williams 🎮Ⓒ , Brian Kam , Eleanor Douglas...",14
1730,team,result,side,adjudicators,15
1731,Copenhagen A,advancing,Opening Government,"Jack PalmerⒸ , Cerys Walsh , Geneva “good chai...",15
1732,Trivium B,eliminated,Closing Opposition,"Jack PalmerⒸ , Cerys Walsh , Geneva “good chai...",15
1733,Belgrade A,eliminated,Closing Government,"Jack PalmerⒸ , Cerys Walsh , Geneva “good chai...",15


In [76]:
df7 = df4
df7['Speaker1'], df7['Speaker2'],df7['Speaker1_score'], df7['Speaker2_score'], df7['Comp_name'] = 0,0,0,0,comp_name
for i in df7.index:
    #print(df7.Rk[i])
    loop_speaker1 = 'iron/swing'
    loop_speaker2 = 'iron/swing'
    loop_speaker1_score = 0
    loop_speaker2_score = 0
    loop_round = df7.Round[i]
    for j in speakertab.index:
        if df7.Rk[i] == speakertab.team[j]:
            if loop_speaker1 == 'iron/swing':
                loop_speaker1 = speakertab.speaker[j]
                if df7.team[i] == '1st' or df7.team[i] == '2nd' or df7.team[i] == '3rd' or df7.team[i] == '4th':
                    loop_speaker1_score = speakertab.iat[j,3+loop_round]
                else:
                    loop_speaker1_score = 'outround'
            else:
                loop_speaker2 = speakertab.speaker[j]
                if df7.team[i] == '1st' or df7.team[i] == '2nd' or df7.team[i] == '3rd' or df7.team[i] == '4th':
                    loop_speaker2_score = speakertab.iat[j,3+loop_round]
                else:
                    loop_speaker2_score = 'outround'
    df7.Speaker1[i] = loop_speaker1
    df7.Speaker2[i] = loop_speaker2
    df7.Speaker1_score[i] = loop_speaker1_score
    df7.Speaker2_score[i] = loop_speaker2_score
df7

C:\Users\andra\AppData\Local\Temp\ipykernel_2496\61731358.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df7.Speaker1[i] = loop_speaker1
C:\Users\andra\AppData\Local\Temp\ipykernel_2496\61731358.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df7.Speaker2[i] = loop_speaker2
C:\Users\andra\AppData\Local\Temp\ipykernel_2496\61731358.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df7.Speaker1_score[i] = loop_speaker1_scor

,Rk,team,categories,R1,Round,Speaker1,Speaker2,Speaker1_score,Speaker2_score,Comp_name
0,team,result,side,ballot,1,iron/swing,iron/swing,0,0,EUDC2024
1,Munich A,1st,Closing Opposition,"Idan BresslerⒸ , Olivia Ortlieb , AnaⓉ",1,Philipp Steen,Uzair Tajuddin,80,81,EUDC2024
2,LSE A,3rd,Opening Government,"Idan BresslerⒸ , Olivia Ortlieb , AnaⓉ",1,Deon Kiew,Tanae Rao,78,77,EUDC2024
3,TCD Phil C,4th,Closing Government,"Idan BresslerⒸ , Olivia Ortlieb , AnaⓉ",1,Vikram Balasubramanian,Raluca Bleier,76,74,EUDC2024
4,Zagreb A,2nd,Opening Opposition,"Idan BresslerⒸ , Olivia Ortlieb , AnaⓉ",1,Petar Žnidar,Jagoda Sabljić,79,78,EUDC2024
...,...,...,...,...,...,...,...,...,...,...
1739,team,result,side,adjudicators,16,iron/swing,iron/swing,0,0,EUDC2024
1740,Copenhagen A,eliminated,Opening Government,"Ilija IvaniševićⒸ , Alishba Irfan , Andrew Che...",16,Laura Serafine Pilmark,Martha Johanne Schou,outround,outround,EUDC2024
1741,LSE B,eliminated,Closing Government,"Ilija IvaniševićⒸ , Alishba Irfan , Andrew Che...",16,Redacted (Expelled),Sachin Tissera,outround,outround,EUDC2024
1742,Tel Aviv A,eliminated,Opening Opposition,"Ilija IvaniševićⒸ , Alishba Irfan , Andrew Che...",16,Annie Schwartz,Boaz Katzir,outround,outround,EUDC2024


## IV. Grouping the teambased result data up to debate based result data

In [84]:
df8 = df7
df8

grouped = df8.groupby(['Round', 'R1']).agg(list).reset_index()

# Flatten the grouped DataFrame by splitting the lists into separate columns
# Here, we use the `apply(pd.Series)` to convert each list into a separate column
grouped_expanded = pd.concat([grouped[['Round', 'R1']], 
                              grouped['Rk'].apply(pd.Series), 
                              grouped['categories'].apply(pd.Series), 
                              grouped['team'].apply(pd.Series),
                              grouped['Speaker1'].apply(pd.Series),
                              grouped['Speaker2'].apply(pd.Series),
                              grouped['Speaker1_score'].apply(pd.Series),
                              grouped['Speaker2_score'].apply(pd.Series)], axis=1)

# Rename the columns to match the desired output


In [89]:
grouped_expanded.columns = ['Round', 'Panel', 
                            'Team1', 'Team2', 'Team3', 'Team4',
                            'Team1 position', 'Team2 position', 'Team3 position', 'Team4 position',
                            'Team1 result', 'Team2 result', 'Team3 result', 'Team4 result',
                            'T1_S1 name', 'T2_S1 name', 'T3_S1 name', 'T4_S1 name',
                            'T1_S2 name', 'T2_S2 name', 'T3_S2 name', 'T4_S2 name',
                            'T1_S1 score', 'T2_S1 score', 'T3_S1 score', 'T4_S1 score',
                            'T1_S2 score', 'T2_S2 score', 'T3_S2 score', 'T4_S2 score']

In [90]:
grouped_expanded

,Round,Panel,Team1,Team2,Team3,Team4,Team1 position,Team2 position,Team3 position,Team4 position,...,T3_S2 name,T4_S2 name,T1_S1 score,T2_S1 score,T3_S1 score,T4_S1 score,T1_S2 score,T2_S2 score,T3_S2 score,T4_S2 score
0,1,"Alishba IrfanⒸ , Daniël P. Hooimeijer , Daniel...",Copenhagen B,GUU B,EDS A,TUT B,Opening Government,Opening Opposition,Closing Opposition,Closing Government,...,Aniket Godbole,Sofia Larens,77,80,82,73,74,80,80,72
1,1,"Ana ComanⒸ , Zsombor Kiss , Ari BⓉ",COLMAN OA,LSE B,TCD Phil A,Manchester A,Closing Government,Closing Opposition,Opening Opposition,Opening Government,...,Oisín Browne,Wajeeh Maaz,74,79,80,78,74,79,79,79
2,1,"Andrew ChenⒸ , Anna Neuber , Ron EdenⓉ",Durham B,KCL A,Tel Aviv A,Zagreb E,Opening Government,Opening Opposition,Closing Government,Closing Opposition,...,Boaz Katzir,Sara Krpan,79,82,83,75,79,81,82,75
3,1,Anna “running away from personal feedback” Shr...,Imperial C,OUI TI,Oxford C,Ljubljana A,Opening Government,Opening Opposition,Closing Government,Closing Opposition,...,Gareth Lim,Teja Sapač,72,75,73,70,71,75,73,70
4,1,"Assaph HananyⒸ , Alyssa Mills , Luisa JacobiⓉ",Karlsruhe A,Cardiff A,Oxford D,Utrecht A,Opening Government,Opening Opposition,Closing Opposition,Closing Government,...,Tanush Agarwal,Roald de Vries,75,78,75,74,73,77,76,73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,15,"Andrew ChenⒸ , Ana Coman , Jack Williams 🎮 , J...",Munich A,LSE B,Tel Aviv A,Athens A,Opening Opposition,Closing Opposition,Closing Government,Opening Government,...,Boaz Katzir,George Fragkiadakis,outround,outround,outround,outround,outround,outround,outround,outround
444,15,"Jack PalmerⒸ , Cerys Walsh , Geneva “good chai...",Copenhagen A,Trivium B,Belgrade A,Zagreb A,Opening Government,Closing Opposition,Closing Government,Opening Opposition,...,Miloš Prokić,Jagoda Sabljić,outround,outround,outround,outround,outround,outround,outround,outround
445,15,adjudicators,team,NaN,NaN,NaN,side,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
446,16,"Ilija IvaniševićⒸ , Alishba Irfan , Andrew Che...",Copenhagen A,LSE B,Tel Aviv A,Zagreb A,Opening Government,Closing Government,Opening Opposition,Closing Opposition,...,Boaz Katzir,Jagoda Sabljić,outround,outround,outround,outround,outround,outround,outround,outround


In [91]:
df9 = grouped_expanded
df9

,Round,Panel,Team1,Team2,Team3,Team4,Team1 position,Team2 position,Team3 position,Team4 position,...,T3_S2 name,T4_S2 name,T1_S1 score,T2_S1 score,T3_S1 score,T4_S1 score,T1_S2 score,T2_S2 score,T3_S2 score,T4_S2 score
0,1,"Alishba IrfanⒸ , Daniël P. Hooimeijer , Daniel...",Copenhagen B,GUU B,EDS A,TUT B,Opening Government,Opening Opposition,Closing Opposition,Closing Government,...,Aniket Godbole,Sofia Larens,77,80,82,73,74,80,80,72
1,1,"Ana ComanⒸ , Zsombor Kiss , Ari BⓉ",COLMAN OA,LSE B,TCD Phil A,Manchester A,Closing Government,Closing Opposition,Opening Opposition,Opening Government,...,Oisín Browne,Wajeeh Maaz,74,79,80,78,74,79,79,79
2,1,"Andrew ChenⒸ , Anna Neuber , Ron EdenⓉ",Durham B,KCL A,Tel Aviv A,Zagreb E,Opening Government,Opening Opposition,Closing Government,Closing Opposition,...,Boaz Katzir,Sara Krpan,79,82,83,75,79,81,82,75
3,1,Anna “running away from personal feedback” Shr...,Imperial C,OUI TI,Oxford C,Ljubljana A,Opening Government,Opening Opposition,Closing Government,Closing Opposition,...,Gareth Lim,Teja Sapač,72,75,73,70,71,75,73,70
4,1,"Assaph HananyⒸ , Alyssa Mills , Luisa JacobiⓉ",Karlsruhe A,Cardiff A,Oxford D,Utrecht A,Opening Government,Opening Opposition,Closing Opposition,Closing Government,...,Tanush Agarwal,Roald de Vries,75,78,75,74,73,77,76,73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,15,"Andrew ChenⒸ , Ana Coman , Jack Williams 🎮 , J...",Munich A,LSE B,Tel Aviv A,Athens A,Opening Opposition,Closing Opposition,Closing Government,Opening Government,...,Boaz Katzir,George Fragkiadakis,outround,outround,outround,outround,outround,outround,outround,outround
444,15,"Jack PalmerⒸ , Cerys Walsh , Geneva “good chai...",Copenhagen A,Trivium B,Belgrade A,Zagreb A,Opening Government,Closing Opposition,Closing Government,Opening Opposition,...,Miloš Prokić,Jagoda Sabljić,outround,outround,outround,outround,outround,outround,outround,outround
445,15,adjudicators,team,NaN,NaN,NaN,side,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
446,16,"Ilija IvaniševićⒸ , Alishba Irfan , Andrew Che...",Copenhagen A,LSE B,Tel Aviv A,Zagreb A,Opening Government,Closing Government,Opening Opposition,Closing Opposition,...,Boaz Katzir,Jagoda Sabljić,outround,outround,outround,outround,outround,outround,outround,outround


In [92]:
df9 = df9.drop(df9[df9['Team1'] == 'team'].index)
df9 = df9.reset_index()
df9 = df9.drop('index', axis=1)
df9

,Round,Panel,Team1,Team2,Team3,Team4,Team1 position,Team2 position,Team3 position,Team4 position,...,T3_S2 name,T4_S2 name,T1_S1 score,T2_S1 score,T3_S1 score,T4_S1 score,T1_S2 score,T2_S2 score,T3_S2 score,T4_S2 score
0,1,"Alishba IrfanⒸ , Daniël P. Hooimeijer , Daniel...",Copenhagen B,GUU B,EDS A,TUT B,Opening Government,Opening Opposition,Closing Opposition,Closing Government,...,Aniket Godbole,Sofia Larens,77,80,82,73,74,80,80,72
1,1,"Ana ComanⒸ , Zsombor Kiss , Ari BⓉ",COLMAN OA,LSE B,TCD Phil A,Manchester A,Closing Government,Closing Opposition,Opening Opposition,Opening Government,...,Oisín Browne,Wajeeh Maaz,74,79,80,78,74,79,79,79
2,1,"Andrew ChenⒸ , Anna Neuber , Ron EdenⓉ",Durham B,KCL A,Tel Aviv A,Zagreb E,Opening Government,Opening Opposition,Closing Government,Closing Opposition,...,Boaz Katzir,Sara Krpan,79,82,83,75,79,81,82,75
3,1,Anna “running away from personal feedback” Shr...,Imperial C,OUI TI,Oxford C,Ljubljana A,Opening Government,Opening Opposition,Closing Government,Closing Opposition,...,Gareth Lim,Teja Sapač,72,75,73,70,71,75,73,70
4,1,"Assaph HananyⒸ , Alyssa Mills , Luisa JacobiⓉ",Karlsruhe A,Cardiff A,Oxford D,Utrecht A,Opening Government,Opening Opposition,Closing Opposition,Closing Government,...,Tanush Agarwal,Roald de Vries,75,78,75,74,73,77,76,73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,14,"Marina KojićⒸ , Jack Palmer , Tom Kuson , Vlad...",Copenhagen A,HUJI AA,Trivium B,Tartu A,Closing Opposition,Closing Government,Opening Government,Opening Opposition,...,Adrianna Mazur,Tanel Mütt,outround,outround,outround,outround,outround,outround,outround,outround
428,14,"ValeriuⒸ , Aymaan Sheikh , Gil Peled , Kayla P...",Munich B,Belgrade B,Belgrade A,Zagreb A,Closing Opposition,Closing Government,Opening Opposition,Opening Government,...,Miloš Prokić,Jagoda Sabljić,outround,outround,outround,outround,outround,outround,outround,outround
429,15,"Andrew ChenⒸ , Ana Coman , Jack Williams 🎮 , J...",Munich A,LSE B,Tel Aviv A,Athens A,Opening Opposition,Closing Opposition,Closing Government,Opening Government,...,Boaz Katzir,George Fragkiadakis,outround,outround,outround,outround,outround,outround,outround,outround
430,15,"Jack PalmerⒸ , Cerys Walsh , Geneva “good chai...",Copenhagen A,Trivium B,Belgrade A,Zagreb A,Opening Government,Closing Opposition,Closing Government,Opening Opposition,...,Miloš Prokić,Jagoda Sabljić,outround,outround,outround,outround,outround,outround,outround,outround


In [20]:
df5 = df4
df5

grouped = df5.groupby(['Round', 'R1']).agg(list).reset_index()

# Flatten the grouped DataFrame by splitting the lists into separate columns
# Here, we use the `apply(pd.Series)` to convert each list into a separate column
grouped_expanded = pd.concat([grouped[['Round', 'R1']], 
                              grouped['Rk'].apply(pd.Series), 
                              grouped['categories'].apply(pd.Series), 
                              grouped['team'].apply(pd.Series)], axis=1)

# Rename the columns to match the desired output
grouped_expanded.columns = ['Round', 'Panel', 
                            'Team1', 'Team2', 'Team3', 'Team4',
                            'Team1 position', 'Team2 position', 'Team3 position', 'Team4 position',
                            'Team1 result', 'Team2 result', 'Team3 result', 'Team4 result']


In [21]:
grouped_expanded

,Round,Panel,Team1,Team2,Team3,Team4,Team1 position,Team2 position,Team3 position,Team4 position,Team1 result,Team2 result,Team3 result,Team4 result
0,1,"Alishba IrfanⒸ , Daniël P. Hooimeijer , Daniel...",Copenhagen B,GUU B,EDS A,TUT B,Opening Government,Opening Opposition,Closing Opposition,Closing Government,3rd,2nd,1st,4th
1,1,"Ana ComanⒸ , Zsombor Kiss , Ari BⓉ",COLMAN OA,LSE B,TCD Phil A,Manchester A,Closing Government,Closing Opposition,Opening Opposition,Opening Government,4th,2nd,1st,3rd
2,1,"Andrew ChenⒸ , Anna Neuber , Ron EdenⓉ",Durham B,KCL A,Tel Aviv A,Zagreb E,Opening Government,Opening Opposition,Closing Government,Closing Opposition,3rd,2nd,1st,4th
3,1,Anna “running away from personal feedback” Shr...,Imperial C,OUI TI,Oxford C,Ljubljana A,Opening Government,Opening Opposition,Closing Government,Closing Opposition,3rd,1st,2nd,4th
4,1,"Assaph HananyⒸ , Alyssa Mills , Luisa JacobiⓉ",Karlsruhe A,Cardiff A,Oxford D,Utrecht A,Opening Government,Opening Opposition,Closing Opposition,Closing Government,3rd,1st,2nd,4th
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,15,"Andrew ChenⒸ , Ana Coman , Jack Williams 🎮 , J...",Munich A,LSE B,Tel Aviv A,Athens A,Opening Opposition,Closing Opposition,Closing Government,Opening Government,eliminated,advancing,advancing,eliminated
444,15,"Jack PalmerⒸ , Cerys Walsh , Geneva “good chai...",Copenhagen A,Trivium B,Belgrade A,Zagreb A,Opening Government,Closing Opposition,Closing Government,Opening Opposition,advancing,eliminated,eliminated,advancing
445,15,adjudicators,team,NaN,NaN,NaN,side,NaN,NaN,NaN,result,NaN,NaN,NaN
446,16,"Ilija IvaniševićⒸ , Alishba Irfan , Andrew Che...",Copenhagen A,LSE B,Tel Aviv A,Zagreb A,Opening Government,Closing Government,Opening Opposition,Closing Opposition,eliminated,eliminated,eliminated,advancing


In [22]:
df6 = grouped_expanded
df6

,Round,Panel,Team1,Team2,Team3,Team4,Team1 position,Team2 position,Team3 position,Team4 position,Team1 result,Team2 result,Team3 result,Team4 result
0,1,"Alishba IrfanⒸ , Daniël P. Hooimeijer , Daniel...",Copenhagen B,GUU B,EDS A,TUT B,Opening Government,Opening Opposition,Closing Opposition,Closing Government,3rd,2nd,1st,4th
1,1,"Ana ComanⒸ , Zsombor Kiss , Ari BⓉ",COLMAN OA,LSE B,TCD Phil A,Manchester A,Closing Government,Closing Opposition,Opening Opposition,Opening Government,4th,2nd,1st,3rd
2,1,"Andrew ChenⒸ , Anna Neuber , Ron EdenⓉ",Durham B,KCL A,Tel Aviv A,Zagreb E,Opening Government,Opening Opposition,Closing Government,Closing Opposition,3rd,2nd,1st,4th
3,1,Anna “running away from personal feedback” Shr...,Imperial C,OUI TI,Oxford C,Ljubljana A,Opening Government,Opening Opposition,Closing Government,Closing Opposition,3rd,1st,2nd,4th
4,1,"Assaph HananyⒸ , Alyssa Mills , Luisa JacobiⓉ",Karlsruhe A,Cardiff A,Oxford D,Utrecht A,Opening Government,Opening Opposition,Closing Opposition,Closing Government,3rd,1st,2nd,4th
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,15,"Andrew ChenⒸ , Ana Coman , Jack Williams 🎮 , J...",Munich A,LSE B,Tel Aviv A,Athens A,Opening Opposition,Closing Opposition,Closing Government,Opening Government,eliminated,advancing,advancing,eliminated
444,15,"Jack PalmerⒸ , Cerys Walsh , Geneva “good chai...",Copenhagen A,Trivium B,Belgrade A,Zagreb A,Opening Government,Closing Opposition,Closing Government,Opening Opposition,advancing,eliminated,eliminated,advancing
445,15,adjudicators,team,NaN,NaN,NaN,side,NaN,NaN,NaN,result,NaN,NaN,NaN
446,16,"Ilija IvaniševićⒸ , Alishba Irfan , Andrew Che...",Copenhagen A,LSE B,Tel Aviv A,Zagreb A,Opening Government,Closing Government,Opening Opposition,Closing Opposition,eliminated,eliminated,eliminated,advancing


In [23]:
df6 = df6.drop(df6[df6['Team1'] == 'team'].index)
df6 = df6.reset_index()
df6 = df6.drop('index', axis=1)
df6

,index,Round,Panel,Team1,Team2,Team3,Team4,Team1 position,Team2 position,Team3 position,Team4 position,Team1 result,Team2 result,Team3 result,Team4 result
0,0,1,"Alishba IrfanⒸ , Daniël P. Hooimeijer , Daniel...",Copenhagen B,GUU B,EDS A,TUT B,Opening Government,Opening Opposition,Closing Opposition,Closing Government,3rd,2nd,1st,4th
1,1,1,"Ana ComanⒸ , Zsombor Kiss , Ari BⓉ",COLMAN OA,LSE B,TCD Phil A,Manchester A,Closing Government,Closing Opposition,Opening Opposition,Opening Government,4th,2nd,1st,3rd
2,2,1,"Andrew ChenⒸ , Anna Neuber , Ron EdenⓉ",Durham B,KCL A,Tel Aviv A,Zagreb E,Opening Government,Opening Opposition,Closing Government,Closing Opposition,3rd,2nd,1st,4th
3,3,1,Anna “running away from personal feedback” Shr...,Imperial C,OUI TI,Oxford C,Ljubljana A,Opening Government,Opening Opposition,Closing Government,Closing Opposition,3rd,1st,2nd,4th
4,4,1,"Assaph HananyⒸ , Alyssa Mills , Luisa JacobiⓉ",Karlsruhe A,Cardiff A,Oxford D,Utrecht A,Opening Government,Opening Opposition,Closing Opposition,Closing Government,3rd,1st,2nd,4th
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,440,14,"Marina KojićⒸ , Jack Palmer , Tom Kuson , Vlad...",Copenhagen A,HUJI AA,Trivium B,Tartu A,Closing Opposition,Closing Government,Opening Government,Opening Opposition,advancing,eliminated,advancing,eliminated
428,441,14,"ValeriuⒸ , Aymaan Sheikh , Gil Peled , Kayla P...",Munich B,Belgrade B,Belgrade A,Zagreb A,Closing Opposition,Closing Government,Opening Opposition,Opening Government,eliminated,eliminated,advancing,advancing
429,443,15,"Andrew ChenⒸ , Ana Coman , Jack Williams 🎮 , J...",Munich A,LSE B,Tel Aviv A,Athens A,Opening Opposition,Closing Opposition,Closing Government,Opening Government,eliminated,advancing,advancing,eliminated
430,444,15,"Jack PalmerⒸ , Cerys Walsh , Geneva “good chai...",Copenhagen A,Trivium B,Belgrade A,Zagreb A,Opening Government,Closing Opposition,Closing Government,Opening Opposition,advancing,eliminated,eliminated,advancing


In [46]:
df7 = df6
df7['Speaker1'], df7['Speaker2'],df7['Speaker1_score'], df7['Speaker2_score'], df7['Comp_name'] = 0,0,0,0,comp_name
for i in df7.index:
    loop_speaker1 = 'iron/swing'
    loop_speaker2 = 'iron/swing'
    for j in speakertab.index:
        if df7.Team1[i] == speakertab.team[j]:
            print(df7.Team1[i])
df7

Copenhagen B
Copenhagen B
COLMAN OA
COLMAN OA
Durham B
Durham B
Imperial C
Imperial C
Karlsruhe A
Karlsruhe A
Cicero DG
Cicero DG
Copenhagen A
Copenhagen A
Bar-Ilan LH
Bar-Ilan LH
AUEB B
AUEB B
TCD Hist AM
TCD Hist AM
Aberystwyth A
Aberystwyth A
Reichman B
Reichman B
Amsterdam RZ
Amsterdam RZ
Imperial A
Imperial A
GUU A
GUU A
CHU A
CHU A
OUI DL
OUI DL
Munich A
Munich A
Vienna B
Vienna B
Liverpool B
Liverpool B
AUEB A
AUEB A
Amsterdam EE
Amsterdam EE
Reichman C
Reichman C
Cardiff B
Cardiff B
ACG A
ACG A
Cambridge A
Cambridge A
Sciences Po A
Sciences Po A
Amsterdam OI
Amsterdam OI
BGU A
BGU A
Aberdeen WB
Aberdeen WB
EUDU DM
EUDU DM
Aberystwyth B
Aberystwyth B
BGU D
BGU D
Amsterdam AB
Amsterdam AB
Oxford G
Oxford G
MCC LW
MCC LW
KCL B
KCL B
Berlin A
Berlin A
EUDU FW
EUDU FW
TCD Phil B
TCD Phil B
Liverpool A
Liverpool A
UAM A
UAM A
Athens A
Athens A
EUDU BF
EUDU BF
Bar-Ilan ZT
Bar-Ilan ZT
BGU C
BGU C
AUEB A
AUEB A
KCL B
KCL B
Amsterdam AB
Amsterdam AB
Copenhagen C
Copenhagen C
GUU D
GUU D


,Round,Panel,Team1,Team2,Team3,Team4,Team1 position,Team2 position,Team3 position,Team4 position,Team1 result,Team2 result,Team3 result,Team4 result,Speaker1,Speaker2,Comp_name,Speaker1_score,Speaker2_score
0,1,"Alishba IrfanⒸ , Daniël P. Hooimeijer , Daniel...",Copenhagen B,GUU B,EDS A,TUT B,Opening Government,Opening Opposition,Closing Opposition,Closing Government,3rd,2nd,1st,4th,0,0,EUDC2024,0,0
1,1,"Ana ComanⒸ , Zsombor Kiss , Ari BⓉ",COLMAN OA,LSE B,TCD Phil A,Manchester A,Closing Government,Closing Opposition,Opening Opposition,Opening Government,4th,2nd,1st,3rd,0,0,EUDC2024,0,0
2,1,"Andrew ChenⒸ , Anna Neuber , Ron EdenⓉ",Durham B,KCL A,Tel Aviv A,Zagreb E,Opening Government,Opening Opposition,Closing Government,Closing Opposition,3rd,2nd,1st,4th,0,0,EUDC2024,0,0
3,1,Anna “running away from personal feedback” Shr...,Imperial C,OUI TI,Oxford C,Ljubljana A,Opening Government,Opening Opposition,Closing Government,Closing Opposition,3rd,1st,2nd,4th,0,0,EUDC2024,0,0
4,1,"Assaph HananyⒸ , Alyssa Mills , Luisa JacobiⓉ",Karlsruhe A,Cardiff A,Oxford D,Utrecht A,Opening Government,Opening Opposition,Closing Opposition,Closing Government,3rd,1st,2nd,4th,0,0,EUDC2024,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,14,"Marina KojićⒸ , Jack Palmer , Tom Kuson , Vlad...",Copenhagen A,HUJI AA,Trivium B,Tartu A,Closing Opposition,Closing Government,Opening Government,Opening Opposition,advancing,eliminated,advancing,eliminated,0,0,EUDC2024,0,0
428,14,"ValeriuⒸ , Aymaan Sheikh , Gil Peled , Kayla P...",Munich B,Belgrade B,Belgrade A,Zagreb A,Closing Opposition,Closing Government,Opening Opposition,Opening Government,eliminated,eliminated,advancing,advancing,0,0,EUDC2024,0,0
429,15,"Andrew ChenⒸ , Ana Coman , Jack Williams 🎮 , J...",Munich A,LSE B,Tel Aviv A,Athens A,Opening Opposition,Closing Opposition,Closing Government,Opening Government,eliminated,advancing,advancing,eliminated,0,0,EUDC2024,0,0
430,15,"Jack PalmerⒸ , Cerys Walsh , Geneva “good chai...",Copenhagen A,Trivium B,Belgrade A,Zagreb A,Opening Government,Closing Opposition,Closing Government,Opening Opposition,advancing,eliminated,eliminated,advancing,0,0,EUDC2024,0,0


In [41]:
#speakertab.speaker[1]
speakertab 

,SpeakRank,speaker,categories,team,R1,R2,R3,R4,R5,R6,R7,R8,R9,Spk
0,1,Hadar Goldberg,ESL,HUJI HY,83,83,84,83,80,85,85,88,85,756
1,2,Martha McKinney-Perry,NaN,TCD Hist CM,79,85,83,81,86,80,84,87,80,745
2,3=,Aidan Woo,NaN,Oxford A,82,83,83,79,83,84,84,84,82,744
3,3=,Petar Žnidar,ESL,Zagreb A,79,82,80,83,85,81,85,82,87,744
4,5,Yishai Binnes,NaN,HUJI HY,82,83,83,82,78,82,83,87,83,743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,366,Matylda Jellonnek,ESL,Utrecht B,75,77,79,74,74,76,—,—,74,529
366,367,Alisa Popova,NaN,Copenhagen B,77,70,74,77,—,80,75,74,—,527
367,368,Speaker 1,NaN,Swing A,—,—,—,76,74,—,—,—,79,229
368,369,Speaker 2,NaN,Swing A,—,—,—,76,66,—,—,—,79,221


## Part V. Formatting the data before saving

In [111]:
# Define column names
columns = ['Round', 'Panel', 
           'OG', 'OO', 'CG','CO',
           'OG speaker 1', 'OG speaker 2', 'OO speaker 1','OO speaker 2',
           'OG speaker 1', 'CG speaker 2', 'CO speaker 1','CO speaker 2',
           'OG1 score', 'OG2 score', 'OO1 score','OO2 score',
           'CG1 score', 'CG2 score', 'CO1 score','CO2 score']

# Get length of df9 
rownumbers = df9.shape[0] 

# Create an empty DataFrame with NaN (or pd.NA) values
data = pd.DataFrame(data={col: [0] * rownumbers for col in columns})

for i in df9.index:
    data.Round[i] = df9.Round[i]
    data.Panel[i] = df9.Panel[i]
    
data

C:\Users\andra\AppData\Local\Temp\ipykernel_2496\2229222553.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Round[i] = df9.Round[i]


,Round,Panel,OG,OO,CG,CO,OG speaker 1,OG speaker 2,OO speaker 1,OO speaker 2,...,CO speaker 1,CO speaker 2,OG1 score,OG2 score,OO1 score,OO2 score,CG1 score,CG2 score,CO1 score,CO2 score
0,1,"Alishba IrfanⒸ , Daniël P. Hooimeijer , Daniel...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,"Ana ComanⒸ , Zsombor Kiss , Ari BⓉ",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,"Andrew ChenⒸ , Anna Neuber , Ron EdenⓉ",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,Anna “running away from personal feedback” Shr...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,"Assaph HananyⒸ , Alyssa Mills , Luisa JacobiⓉ",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,14,"Marina KojićⒸ , Jack Palmer , Tom Kuson , Vlad...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
428,14,"ValeriuⒸ , Aymaan Sheikh , Gil Peled , Kayla P...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
429,15,"Andrew ChenⒸ , Ana Coman , Jack Williams 🎮 , J...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
430,15,"Jack PalmerⒸ , Cerys Walsh , Geneva “good chai...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [108]:
df9

,Round,Panel,Team1,Team2,Team3,Team4,Team1 position,Team2 position,Team3 position,Team4 position,...,T3_S2 name,T4_S2 name,T1_S1 score,T2_S1 score,T3_S1 score,T4_S1 score,T1_S2 score,T2_S2 score,T3_S2 score,T4_S2 score
0,1,"Alishba IrfanⒸ , Daniël P. Hooimeijer , Daniel...",Copenhagen B,GUU B,EDS A,TUT B,Opening Government,Opening Opposition,Closing Opposition,Closing Government,...,Aniket Godbole,Sofia Larens,77,80,82,73,74,80,80,72
1,1,"Ana ComanⒸ , Zsombor Kiss , Ari BⓉ",COLMAN OA,LSE B,TCD Phil A,Manchester A,Closing Government,Closing Opposition,Opening Opposition,Opening Government,...,Oisín Browne,Wajeeh Maaz,74,79,80,78,74,79,79,79
2,1,"Andrew ChenⒸ , Anna Neuber , Ron EdenⓉ",Durham B,KCL A,Tel Aviv A,Zagreb E,Opening Government,Opening Opposition,Closing Government,Closing Opposition,...,Boaz Katzir,Sara Krpan,79,82,83,75,79,81,82,75
3,1,Anna “running away from personal feedback” Shr...,Imperial C,OUI TI,Oxford C,Ljubljana A,Opening Government,Opening Opposition,Closing Government,Closing Opposition,...,Gareth Lim,Teja Sapač,72,75,73,70,71,75,73,70
4,1,"Assaph HananyⒸ , Alyssa Mills , Luisa JacobiⓉ",Karlsruhe A,Cardiff A,Oxford D,Utrecht A,Opening Government,Opening Opposition,Closing Opposition,Closing Government,...,Tanush Agarwal,Roald de Vries,75,78,75,74,73,77,76,73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,14,"Marina KojićⒸ , Jack Palmer , Tom Kuson , Vlad...",Copenhagen A,HUJI AA,Trivium B,Tartu A,Closing Opposition,Closing Government,Opening Government,Opening Opposition,...,Adrianna Mazur,Tanel Mütt,outround,outround,outround,outround,outround,outround,outround,outround
428,14,"ValeriuⒸ , Aymaan Sheikh , Gil Peled , Kayla P...",Munich B,Belgrade B,Belgrade A,Zagreb A,Closing Opposition,Closing Government,Opening Opposition,Opening Government,...,Miloš Prokić,Jagoda Sabljić,outround,outround,outround,outround,outround,outround,outround,outround
429,15,"Andrew ChenⒸ , Ana Coman , Jack Williams 🎮 , J...",Munich A,LSE B,Tel Aviv A,Athens A,Opening Opposition,Closing Opposition,Closing Government,Opening Government,...,Boaz Katzir,George Fragkiadakis,outround,outround,outround,outround,outround,outround,outround,outround
430,15,"Jack PalmerⒸ , Cerys Walsh , Geneva “good chai...",Copenhagen A,Trivium B,Belgrade A,Zagreb A,Opening Government,Closing Opposition,Closing Government,Opening Opposition,...,Miloš Prokić,Jagoda Sabljić,outround,outround,outround,outround,outround,outround,outround,outround
